In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'      'Internship-Aug2020-Gowtham R.pdf'   Photos
'Getting started.pdf'  'My Drive'			   'Pongal 2021'


In [2]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project

/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project


In [3]:
from keras.models import load_model
import pandas as pd
import numpy as np 
from os.path import join
from myFunctions.candle_stick_plot import plotCandleStick
from myFunctions.list_data_dir import load_data_files
from myFunctions import dataConversion 
from myFunctions.predictionPlot import prediction_plot

In [4]:
fields = ['Date','High','Low','Open','Close']
lookback = 30
datafolder = r"/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/Data/NIFTY50"
prediction_date = pd.Timestamp(2020,11,30,00,00,00)

In [5]:
df = pd.read_csv(r"/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/Data/NIFTY50/ADANIPORTS.csv",usecols = fields)
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Open,High,Low,Close
0,2007-11-27,770.00,1050.00,770.0,962.90
1,2007-11-28,984.00,990.00,874.0,893.90
2,2007-11-29,909.00,914.75,841.0,884.20
3,2007-11-30,890.00,958.00,890.0,921.55
4,2007-12-03,939.75,995.00,922.0,969.30


In [6]:
Low_X , Low_y = dataConversion.convert2matrix(df['Low'],lookback)

In [7]:
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [8]:
Low_X_train , Low_x_test, Low_y_train, Low_y_test = train_test_split(Low_X,Low_y,test_size = 0.2)

In [10]:
from keras.wrappers.scikit_learn import KerasRegressor,KerasClassifier
from sklearn.model_selection import RandomizedSearchCV,KFold

In [11]:
def create_model(layers=1 ,neurons=256):
    model=Sequential()

    model.add(Dense(500,activation='relu',input_shape=(lookback,)))

    for i in range(layers):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='relu'))
    model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])
    return model

In [13]:
model = KerasRegressor(build_fn=create_model,epochs=100,batch_size=16)
params = dict(layers=[1,2,3,4,5],neurons=[5,25,50,100,150,200,250,300])
random_search = RandomizedSearchCV(model,params,cv=KFold(3))
random_search.fit(Low_X_train,Low_y_train)

Streaming output truncated to the last 5000 lines.
Epoch 13/100
107/107 [==============================] - 0s 3ms/step - loss: 128718.7572 - mse: 128718.7572
Epoch 14/100
107/107 [==============================] - 0s 3ms/step - loss: 132357.5578 - mse: 132357.5578
Epoch 15/100
107/107 [==============================] - 0s 3ms/step - loss: 142486.5332 - mse: 142486.5332
Epoch 16/100
107/107 [==============================] - 0s 3ms/step - loss: 136390.6480 - mse: 136390.6480
Epoch 17/100
107/107 [==============================] - 0s 3ms/step - loss: 130423.1856 - mse: 130423.1856
Epoch 18/100
107/107 [==============================] - 0s 3ms/step - loss: 137077.8354 - mse: 137077.8354
Epoch 19/100
107/107 [==============================] - 0s 3ms/step - loss: 137892.5917 - mse: 137892.5917
Epoch 20/100
107/107 [==============================] - 0s 3ms/step - loss: 141368.9944 - mse: 141368.9944
Epoch 21/100
107/107 [==============================] - 0s 3ms/step - loss: 134447.8016 - mse

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fe4aa325750>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'layers': [1, 2, 3, 4, 5],
                                        'neurons': [5, 25, 50, 100, 150, 200,
                                                    250, 300]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [14]:
random_search.best_params_

{'layers': 4, 'neurons': 250}

In [15]:
random_search.best_score_

-413.0830434163411

In [16]:
best_low_model = random_search.best_estimator_

In [17]:
best_low_model.score(Low_x_test,Low_y_test)

40/40 [==============================] - 0s 2ms/step - loss: 337.1225 - mse: 337.1225


-337.1224670410156

In [ ]:
best_low_model.model.evaluate(Low_x_test,Low_y_test)

20/20 [==============================] - 0s 2ms/step - loss: 10392.3135 - mse: 10392.3135


[10392.3134765625, 10392.3134765625]

In [18]:
best_low_model.model.summary()

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_228 (Dense)            (None, 500)               15500     
_________________________________________________________________
dense_229 (Dense)            (None, 250)               125250    
_________________________________________________________________
dense_230 (Dense)            (None, 250)               62750     
_________________________________________________________________
dense_231 (Dense)            (None, 250)               62750     
_________________________________________________________________
dense_232 (Dense)            (None, 250)               62750     
_________________________________________________________________
dense_233 (Dense)            (None, 1)                 251       
Total params: 329,251
Trainable params: 329,251
Non-trainable params: 0
_______________________________________________

In [ ]:
best_low_model.model.evaluate(Low_x_test,Low_y_test)

20/20 [==============================] - 0s 2ms/step - loss: 10392.3135 - mse: 10392.3135


[10392.3134765625, 10392.3134765625]

In [ ]:
Low_x_test[0].reshape(-1,30).shape

(1, 30)

In [19]:
low_predict = best_low_model.predict(Low_x_test)

In [25]:
best_low_model.model.save('/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/models/new_low_model')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/models/new_low_model/assets


In [21]:
from sklearn.metrics import r2_score
r2_score(Low_y_test,low_predict)

0.987455004597792